In [ ]:
!pip install ultralytics

In [ ]:
import zipfile
import os
import shutil
from ultralytics          import YOLO
from google.colab         import drive
import random
import tqdm

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/yolo_dataset.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/coco-seg_5000samples.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
img_dir = '/content/coco-images'
labels_dir = '/content/coco_annotation'
dst_dir = '/content/yolo_dataset'
def add_coco_dataset(img_dir, labels_dir, dst_dir, train_size=0.7, val_size=0.2, test_size=0.1):
  random.seed(42)
  imgs = os.listdir(img_dir)
  random.shuffle(imgs)
  length = len(imgs)

  train_end = int(train_size * length)
  val_end = train_end + int(val_size * length)

  train_imgs = imgs[:train_end]
  val_imgs = imgs[train_end:val_end]
  test_imgs = imgs[val_end:]

  for img in [img for img in train_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/train'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/train'))
  print(f'{train_end} images have been move.')
  for img in [img for img in val_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/val'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/val'))
  print(f'{val_end-train_end} images have been move.')
  for img in [img for img in test_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/test'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/test'))
  print(f'{length-val_end} images have been move.')

add_coco_dataset(img_dir, labels_dir, dst_dir)

3500 images have been move.
1000 images have been move.
500 images have been move.


In [ ]:
# Load a model
model = YOLO('yolov8s-seg.yaml')  # build a new model from YAML
model = YOLO('yolov8s-seg.pt')  # load a pretrained model (recommended for training)
model.info()

100%|██████████| 22.8M/22.8M [00:00<00:00, 68.1MB/s]


YOLOv8s-seg summary: 261 layers, 11821056 parameters, 0 gradients, 42.9 GFLOPs


(261, 11821056, 0, 42.860595200000006)

In [ ]:
# Train the model
model.train(data='/content/yolo_dataset/dataset.yaml', epochs=50, patience=10, imgsz=640, freeze=10, verbose=True, batch=-1, exist_ok=True,
      project='/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models', name='yolo_v8_coco/segment')

Ultralytics YOLOv8.2.14 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/yolo_dataset/dataset.yaml, epochs=50, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models, name=segment, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fal

100%|██████████| 755k/755k [00:00<00:00, 10.9MB/s]


Overriding model.yaml nc=80 with nc=81

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 6.23M/6.23M [00:00<00:00, 53.2MB/s]


AMP: checks passed ✅
AutoBatch: Computing optimal batch size for imgsz=640
AutoBatch: CUDA:0 (Tesla T4) 14.75G total, 0.26G reserved, 0.13G allocated, 14.36G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    11821443       42.86         0.409         41.39           nan        (1, 3, 640, 640)                    list
    11821443       85.73         0.629         22.12           nan        (2, 3, 640, 640)                    list
    11821443       171.5         1.107         35.88           nan        (4, 3, 640, 640)                    list
    11821443       342.9         2.028         58.07           nan        (8, 3, 640, 640)                    list
    11821443       685.8         3.857         86.91           nan       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 36 for CUDA:0 8.85G/14.75G (60%) ✅


...

      50/50      6.59G      1.018      1.495     0.6353     0.9307         13        640: 100%|██████████| 1405/1405 [17:03<00:00,  1.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 201/201 [03:22<00:00,  1.01s/it]


                   all      14434     103158      0.575      0.379      0.412      0.283      0.563       0.37      0.397      0.246

50 epochs completed in 21.214 hours.
Optimizer stripped from /content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v8_coco/segment/weights/last.pt, 23.9MB
Optimizer stripped from /content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v8_coco/segment/weights/best.pt, 23.9MB

Validating /content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v8_coco/segment/weights/best.pt...
Ultralytics YOLOv8.2.14 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-seg summary (fused): 195 layers, 11810947 parameters, 0 gradients, 42.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 201/201 [03:49<00:00,  1.14s/it]


                   all      14434     103158      0.544      0.395      0.417      0.284      0.525      0.384      0.399      0.249
                person      14434       7589      0.822      0.847      0.886       0.66      0.805      0.829      0.869      0.612
               bicycle      14434         93      0.452       0.31      0.334      0.184      0.373      0.256      0.268      0.109
                   car      14434        451      0.639      0.483      0.489      0.322      0.612      0.463      0.465      0.266
            motorcycle      14434         82        0.6      0.415      0.471      0.236      0.547      0.378      0.396      0.192
              airplane      14434         50       0.67       0.56      0.587      0.359      0.598        0.5      0.506        0.3
                   bus      14434         69      0.548      0.522      0.508      0.383      0.517      0.493      0.472      0.329
                 train      14434         56      0.669      0.464   

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78,
       79, 80])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ab5f4ff9e70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v8_coco/segment'

<IPython.core.display.Javascript object>

In [ ]:
model = YOLO('/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v8_coco/segment/weights/best.pt')
model.info()

WARNING ⚠️ /content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9/segment/weights/best.pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 6.0s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv9c-seg summary: 654 layers, 27897891 parameters, 0 gradients, 159.4 GFLOPs


(654, 27897891, 0, 159.4184192)

In [ ]:
# Checking the test set (change from val rute to test rute in yaml file.)
model.val(data='/content/yolo_dataset/dataset.yaml')

In [ ]:
import shutil
shutil.move('/content/runs/segment/val/', '/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v8/segment/test/')

'/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9/segment/test/'

In [ ]:
drive.flush_and_unmount()